<a href="https://colab.research.google.com/github/farahshahhoud/Qlearningconnect4/blob/main/4InRow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from abc import abstractmethod
import os
from pathlib import Path
import keras.models as Km
import keras as K
import numpy as np
import time
import numpy as np
import keras.layers as Kl
from keras import optimizers
from abc import abstractmethod
import random
import numpy as np
import pickle

Using TensorFlow backend.


In [ ]:
class Model:

    def __init__(self,tag):
        self.tag = tag
        self.epsilon = 0.1
        self.alpha = 0.5
        self.gamma = 1
        self.model = self.load_model()

    def load_model(self):
        if self.tag == 1:
           tag = '_first'
        else:
            tag = '_second'''
        s = 'model_values' + tag + '.h5'
        model_file = Path(s)

        if model_file.is_file():
            print('load model:',s)
            model = Km.load_model(s)
        else:
            model = self.create_model()
        return model

    @abstractmethod
    def create_model(self):
        pass

    @abstractmethod
    def state_to_tensor(self, state, move):
        pass

    def calc_value(self, state, move):
        tensor = self.state_to_tensor(state, move)
        value = self.model.predict(tensor)
        # K.backend.clear_session()
        return value

    def calc_target(self, prev_state, prev_move, state, ava_moves, reward):

        v_s = self.calc_value(prev_state, prev_move)

        v = []
        for move in ava_moves:
            v.append(self.calc_value(state, move))

        if reward == 0:
            if len(v) > 0:
                v_s_tag = self.gamma * np.max(v)
            else:
                #print('no moves!!!')
                v_s_tag = 0
            target = np.array(v_s + self.alpha * (reward + v_s_tag - v_s))
        else:
            # v_s_tag = 0
            target = reward

        # target = np.array(v_s + self.alpha * (reward + v_s_tag - v_s))

        # if self.tag == 1:
        #     print('learn general')
        #     print(prev_state, prev_move, state, ava_moves, reward)
        #      print('target: ', target)

        return target

    def train_model(self, prev_state, prev_move, target, epochs):

        tensor = self.state_to_tensor(prev_state, prev_move)

        if target is not None:

            self.model.fit(tensor, target, epochs=epochs, verbose=0)
            K.backend.clear_session()

    def save_model(self):
        if self.tag == 1:
            tag = '_first'
        else:
            tag = '_second'
        s = 'model_values' + tag + '.h5'

        try:
            os.remove(s)
        except:
            pass
        print('model save to: ',s)
        self.model.save(s)

    def learn_batch(self, memory):
        print('start learning player', self.tag)
        print('data length:', len(memory))

        # build x_train
        ind = 0
        x_train = np.zeros((len(memory), 7, 7, 1))
        # x_train = np.zeros((len(memory), 2, 9))
        for v in memory:
            [prev_state, prev_move, _, _, _] = v
            sample = self.state_to_tensor(prev_state, prev_move)
            x_train[ind, :, :, :] = sample
            ind += 1

        # train with planning
        # for i in range(5):
        loss = 20
        count = 0
        while loss > 0.02 and count<20:
            # tic()
            y_train = self.create_targets(memory)
            # toc()
            self.model.fit(x_train, y_train, epochs=5, batch_size=256, verbose=1)
            loss = self.model.evaluate(x_train, y_train, batch_size=256, verbose=0)[0]
            count += 1
            #print('planning number:', count, 'loss', loss)

        loss = self.model.evaluate(x_train, y_train, batch_size=256, verbose=0)
        print('player:', self.tag,'loss: ', loss, 'loops: ', count)

        self.save_model()

    def create_targets(self, memory):
        y_train_ = np.zeros((len(memory), 1))
        count_ = 0
        for v_ in memory:
            [prev_state_, prev_move_, state_, ava_moves_, reward_] = v_
            target = self.calc_target(prev_state_, prev_move_, state_, ava_moves_, reward_)
            y_train_[count_, :] = target
            count_ += 1
            #print('---------')
            #print('player', self.tag)
            #print('prev state', prev_state_)
            #print('prev move', prev_move_)
            #print('state', state_)
            #print('ava moves', ava_moves_)
            #print('reward', reward_)
            #print('target', target)
            
            #value = self.calc_value(prev_state_, prev_move_)
            #print('value through net', value)
            #time.sleep(0.2)

        return y_train_

In [ ]:
class Agent:

    def __init__(self, tag,exploration_factor=1):
        self.tag = tag
        self.exp_factor = exploration_factor
        self.prev_state = np.zeros((6, 7))
        self.prev_move = -1
        self.state = None
        self.move = None
        self.print_value = False
        self.model = Model(self.tag)
        self.memory = []
        self.count_memory = 0
        self.winner_flag = False

    def choose_move(self, state, winner, learn):

        self.load_to_memory(self.prev_state, self.prev_move, state, self.ava_moves(state), self.reward(winner))

        if winner is not None:

            self.count_memory += 1

            self.prev_state = np.zeros((6, 7))
            self.prev_move = -1

            if learn is True and self.count_memory == 1000:
                self.count_memory = 0
                # Offline training
                self.model.learn_batch(self.memory)
                self.memory = []
                # Online training
                #self.learn(self.prev_state, self.prev_move, state, self.ava_moves(state),  -1, self.reward(winner))
            return None

        p = random.uniform(0, 1)

        if p < self.exp_factor:
            idx = self.choose_optimal_move(state)
        else:
            ava_moves = self.ava_moves(state)
            idx = random.choice(ava_moves)

        self.prev_state = state
        self.prev_move = idx

        return idx

    def choose_optimal_move(self, state):

        ava_moves = self.ava_moves(state)
        v = -float('Inf')
        v_list = []
        idx = []
        for move in ava_moves:
            value = self.model.calc_value(state, move)
            v_list.append(round(float(value), 5))

            if value > v:
                v = value
                idx = [move]
            elif v == value:
                idx.append(move)

        idx = random.choice(idx)
        return idx

    def game_winner(self, state):
        winner = None
        for i in range(len(state[:,0])-3):
            for j in range(len(state[0, :])-3):
                winner = self.square_winner(state[i:i+4, j:j+4])
                if winner is not None:
                    # print('winner is:', self.winner)
                    break
            if winner is not None:
                # print('winner is:', self.winner)
                break

        if np.min(np.abs(state[0, :])) != 0:
            winner = 0
            # print('no winner')

        return winner

    @staticmethod
    def square_winner(square):
        s = np.append([np.sum(square, axis=0), np.sum(square, axis=1).T],
                      [np.trace(square), np.flip(square,axis=1).trace()])
        if np.max(s) == 4:
            winner = 1
        elif np.min(s) == -4:
            winner = 2
        else:
            winner = None
        return winner

    @staticmethod
    def make_state_from_move(state, move, player):
        if move is None:
            return state

        state = np.array(state)
        if player == 1:
            tag = 1
        else:
            tag = -1

        if len(np.where(state[:, move] == 0)[0]) == 0:
            print(state)
        idy = np.where(state[:, move] == 0)[0][-1]
        state = np.array(state)
        state[idy, move] = tag

        return state

    def reward(self, winner):

        if winner is self.tag:
            reward = 1
        elif winner is None:
            reward = 0
        elif winner == 0:
            reward = 0.5
        else:
            reward = -1
        return reward

    def learn(self, prev_state, prev_move, state, ava_moves, move, reward):

        if prev_move != -1:

            target = self.model.calc_target(prev_state, prev_move, state, ava_moves, reward)
            # print(target)
            self.model.train_model(prev_state, prev_move, target, 1)

    @abstractmethod
    def ava_moves(self, state):
        pass

    def load_to_memory(self, prev_state, prev_move, state, ava_moves, reward):
        self.memory.append([prev_state, prev_move, state, ava_moves, reward])

    def save_memory(self):
        is_file_ = True
        count = 1
        s = ''
        while is_file_:
            s = 'value_list_' + str(self.tag) + '_' + str(count) + '.pkl'
            if Path(s).is_file():
                is_file_ = True
                count = count + 1
            else:
                is_file_ = False

        with open(s, 'wb') as output:
            pickle.dump(self.memory, output)

In [ ]:
class Game:

    def __init__(self, player1, player2, exp1=1, exp2=1, tag1=1, tag2=2):

        self.players = {1: player1(tag1, exploration_factor=exp1),
                        2: player2(tag2, exploration_factor=exp2)}

        self.state, self.winner, self.turn = self.init_game()
        self.memory = {}

    def play_game(self, learn=False):

        move_count = 0

        while self.winner is None:
            move = self.play_move(learn)

            self.state = self.make_state_from_move(move)
            self.game_winner()
            if self.winner is not None:
              break
            self.next_player()
            move_count += 1

        self.play_move(learn)  
        self.next_player()      
        self.play_move(learn)   
        self.next_player()    

        return self.winner, move_count

    def play_move(self, learn):
        player = self.players[self.turn]
        move=1
        if self.winner is None:
          move = player.choose_move(self.state, self.winner, learn)
        return move

    def play_multiple_games(self, episodes, learn):
        statistics = {1: 0, 2: 0, 0: 0, 'move_count': 0}
        move_count_total = []
        for i in range(episodes):
            winner, move_count = self.play_game(learn)
            move_count_total.append(move_count)
            statistics[winner] = statistics[winner] + 1

            self.state, self.winner, self.turn = self.init_game()

        statistics['move_count'] = np.mean(move_count_total)
        return statistics

    @abstractmethod
    def init_game(self):
        pass

    @abstractmethod
    def make_state_from_move(self, move):
        pass

    @abstractmethod
    def next_player(self):
        pass

    @abstractmethod
    def game_winner(self):
        pass

    @abstractmethod
    def print_game(self):
        pass

In [ ]:
class ConnectFourModel(Model):

    def __init__(self, tag):
        super().__init__(tag)
        pass

    def create_model(self):
        print('new model')

        model = Km.Sequential()
        model.add(Kl.Conv2D(20, (5, 5), padding='same', input_shape=(7, 7, 1)))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(20, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(20, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(30, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(30, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(30, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Conv2D(30, (4, 4), padding='same'))
        model.add(Kl.LeakyReLU(alpha=0.3))

        model.add(Kl.Flatten(input_shape=(7, 7, 1)))
        model.add(Kl.Dense(49))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Dense(7))
        model.add(Kl.LeakyReLU(alpha=0.3))
        model.add(Kl.Dense(7))
        model.add(Kl.LeakyReLU(alpha=0.3))

        model.add(Kl.Dense(1, activation='linear'))
        opt = optimizers.adam()
        model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

        model.summary()

        return model

    def state_to_tensor(self, state, move):

        vec = np.zeros((1, 7))
        if move != -1:
            vec[0, move] = 1
        tensor = np.append(vec, state, axis=0)
        tensor = tensor.reshape((1, 7, 7, 1))

        return tensor

In [ ]:
class ConnectFour(Game):

    def __init__(self, player1, player2, exp1=1, exp2=1, tag1=1, tag2=2):
        super().__init__(player1, player2, exp1, exp2, tag1, tag2)

    def init_game(self):
        return np.zeros((6, 7)), None, 1

    def make_state_from_move(self, move):
        if move is None:
            return self.state

        state = np.array(self.state)
        if self.turn == 1:
            tag = 1
        else:
            tag = -1

        if len(np.where(state[:, move] == 0)[0]) == 0:
            print(state)
        idy = np.where(state[:, move] == 0)[0][-1]
        state = np.array(state)
        state[idy, move] = tag

        return state

    def next_player(self):
        if self.turn == 1:
            self.turn = 2
        else:
            self.turn = 1

    def game_winner(self):
        for i in range(len(self.state[:,0])-3):
            for j in range(len(self.state[0, :])-3):
                self.square_winner(self.state[i:i+4, j:j+4])
                if self.winner is not None:
                    # print('winner is:', self.winner)
                    break
            if self.winner is not None:
                print('winner is:', self.winner)
                break

        if np.min(np.abs(self.state[0, :])) != 0:
            self.winner = 0
            print('no winner')

    def square_winner(self, square):
        s = np.append([np.sum(square, axis=0), np.sum(square, axis=1).T],
                      [np.trace(square), np.flip(square,axis=1).trace()])
        if np.max(s) == 4:
            self.winner = 1
        elif np.min(s) == -4:
            self.winner = 2
        else:
            self.winner = None
        return self.winner

    def print_game(self):

      print('  --------------')
      charar = np.chararray(state.shape,unicode=True)
      for i in range(state.shape[0]):
        for j in range(state.shape[1]):
          if state[i][j]==1: charar[i][j]='*'
          elif state[i][j]==-1: charar[i][j]='-'
          else: charar[i][j]='0'
      print(charar)
      print('  --------------')

In [ ]:
class HumanPlayer:

    def __init__(self, tag, exploration_factor=1):
        self.print_value = False
        self.exp_factor = exploration_factor
        self.tag = tag

    @staticmethod
    def choose_move(state, winner, learn):
        if winner is not None:
          return 1
        print('  --------------')
        charar = np.chararray(state.shape,unicode=True)
        for i in range(state.shape[0]):
          for j in range(state.shape[1]):
            if state[i][j]==1: charar[i][j]='*'
            elif state[i][j]==-1: charar[i][j]='-'
            else: charar[i][j]='0'
        print(charar)
        print('  --------------')
        idx = int(input('Choose move number: ')) - 1
        return idx

    def save_memory(self):
        pass

In [ ]:
class ConnectFourAgent(Agent):

    def __init__(self, tag, exploration_factor=1):

        super().__init__(tag, exploration_factor)
        self.model = ConnectFourModel(tag)
        self.prev_state = np.zeros((6, 7))

    def ava_moves(self, state):
        moves = np.where(state[0, :] == 0)[0]
        return moves

In [ ]:
episodes = 6000

game = ConnectFour(ConnectFourAgent, ConnectFourAgent, 0.7, 0.7)
statistics = game.play_multiple_games(episodes, learn=True)
print('0.7, 0.7', statistics)

new model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 7, 20)          520       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 20)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 20)          6420      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 20)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 20)          6420      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 20)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 30

In [ ]:
episodes = 1

game = ConnectFour( ConnectFourAgent,HumanPlayer, 1,1)
statistics = game.play_multiple_games(episodes, learn=False)

load model: model_values_first.h5
load model: model_values_first.h5
  --------------
[['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']]
  --------------
Choose move number: 3
  --------------
[['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']
 ['0' '0' '-' '*' '0' '0' '0']]
  --------------
Choose move number: 2
  --------------
[['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']
 ['0' '-' '-' '*' '0' '0' '0']]
  --------------
Choose move number: 4
  --------------
[['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '-' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']
 ['0' '0' '0' '*' '0' '0' '0']
 ['0' '-' '-' '*' '*' '0' '0']]